In [5]:
from sqlalchemy import create_engine
import pandas as pd

In [6]:
engine = create_engine('sqlite:///country_club.db')
table_names = engine.table_names()
print(table_names)

['Bookings', 'Facilities', 'Members']


PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions. 

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [18]:
sql = 'SELECT f.name, '
sql += 'SUM(CASE WHEN b.memid =0 '
sql += 'THEN b.slots * f.guestcost '
sql += 'ELSE b.slots * f.membercost '
sql += 'END ) AS revenue '
sql += 'FROM Bookings b '
sql += 'JOIN Facilities f ON b.facid = f.facid '
sql += 'GROUP BY f.name '
sql += 'HAVING revenue < 1000 '
sql += 'ORDER BY revenue'
print(sql)

SELECT f.name, SUM(CASE WHEN b.memid =0 THEN b.slots * f.guestcost ELSE b.slots * f.membercost END ) AS revenue FROM Bookings b JOIN Facilities f ON b.facid = f.facid GROUP BY f.name HAVING revenue < 1000 ORDER BY revenue


In [19]:
with engine.connect() as con:
    rs = con.execute(sql)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
print(df)

            name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270


 Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [20]:
sql = 'select m.surname || ", " || m.firstname as member, '
sql += 'rec.surname ||", " || rec.firstname as recby '
sql += 'from Members m '
sql += 'left join Members rec on m.recommendedby = rec.memid '
sql += 'order by member'
print(sql)

select m.surname || ", " || m.firstname as member, rec.surname ||", " || rec.firstname as recby from Members m left join Members rec on m.recommendedby = rec.memid order by member


In [21]:
with engine.connect() as con:
    rs = con.execute(sql)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
print(df)

                      member               recby
0            Bader, Florence    Stibbons, Ponder
1                Baker, Anne    Stibbons, Ponder
2             Baker, Timothy     Farrell, Jemima
3                Boothe, Tim         Rownam, Tim
4            Butters, Gerald       Smith, Darren
5               Coplin, Joan      Baker, Timothy
6             Crumpet, Erica        Smith, Tracy
7                Dare, Nancy    Joplette, Janice
8             Farrell, David                None
9            Farrell, Jemima                None
10              GUEST, GUEST                None
11          Genting, Matthew     Butters, Gerald
12                Hunt, John  Purview, Millicent
13              Jones, David    Joplette, Janice
14            Jones, Douglas        Jones, David
15          Joplette, Janice       Smith, Darren
16           Mackenzie, Anna       Smith, Darren
17             Owen, Charles       Smith, Darren
18             Pinker, David     Farrell, Jemima
19        Purview, M

Q12: Find the facilities with their usage by member, but not guests

In [22]:
sql = 'SELECT f.name, m.surname || ", " || m.firstname as member, count(slots) as usage '
sql += 'FROM Bookings b '
sql += 'JOIN Facilities f ON b.facid = f.facid '
sql += 'JOIN Members m on b.memid = m.memid '
sql += 'where b.memid != 0 '
sql += 'GROUP BY f.name, m.surname, m.firstname '
sql += 'ORDER BY member'
print(sql)

SELECT f.name, m.surname || ", " || m.firstname as member, count(slots) as usage FROM Bookings b JOIN Facilities f ON b.facid = f.facid JOIN Members m on b.memid = m.memid where b.memid != 0 GROUP BY f.name, m.surname, m.firstname ORDER BY member


In [23]:
with engine.connect() as con:
    rs = con.execute(sql)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
print(df)

                name                    member  usage
0    Badminton Court           Bader, Florence      9
1     Massage Room 2           Bader, Florence      2
2         Pool Table           Bader, Florence     23
3      Snooker Table           Bader, Florence     33
4       Squash Court           Bader, Florence      2
..               ...                       ...    ...
197     Squash Court      Tupperware, Hyacinth      1
198  Badminton Court  Worthington-Smyth, Henry      4
199   Massage Room 1  Worthington-Smyth, Henry      1
200       Pool Table  Worthington-Smyth, Henry     33
201     Table Tennis  Worthington-Smyth, Henry      3

[202 rows x 3 columns]


Q13: Find the facilities usage by month, but not guests

In [28]:
sql = 'SELECT f.name, strftime("%m", date(b.starttime)), count(slots) as usage '
sql += 'FROM Bookings b '
sql += 'JOIN Facilities f ON b.facid = f.facid '
sql += 'JOIN Members m on b.memid = m.memid '
sql += 'where b.memid != 0 '
sql += 'GROUP BY f.name, strftime("%m", date(b.starttime)) '
sql += 'ORDER BY strftime("%m", date(b.starttime))'
print(sql)

SELECT f.name, strftime("%m", date(b.starttime)), count(slots) as usage FROM Bookings b JOIN Facilities f ON b.facid = f.facid JOIN Members m on b.memid = m.memid where b.memid != 0 GROUP BY f.name, strftime("%m", date(b.starttime)) ORDER BY strftime("%m", date(b.starttime))


In [29]:
with engine.connect() as con:
    rs = con.execute(sql)
    df = pd.DataFrame(rs.fetchall())
    df.columns = rs.keys()
    
print(df)

               name strftime("%m", date(b.starttime))  usage
0   Badminton Court                                07     51
1    Massage Room 1                                07     77
2    Massage Room 2                                07      4
3        Pool Table                                07    103
4     Snooker Table                                07     68
5      Squash Court                                07     23
6      Table Tennis                                07     48
7    Tennis Court 1                                07     65
8    Tennis Court 2                                07     41
9   Badminton Court                                08    132
10   Massage Room 1                                08    153
11   Massage Room 2                                08      9
12       Pool Table                                08    272
13    Snooker Table                                08    154
14     Squash Court                                08     85
15     Table Tennis     